https://archive.ics.uci.edu/ml/datasets/Drug+Review+Dataset+%28Drugs.com%29

# Imports and required functions

In [2]:
!pip install texthero
!pip install mapply
# !pip install flair
# !pip install simpletransformers
# !pip install tensorboardx
!pip install torchvision 
!pip install pandas -U
!pip install fast-bert

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached texthero-1.1.0-py3-none-any.whl (24 kB)
  Using cached wordcloud-1.8.1-cp37-cp37m-manylinux1_x86_64.whl (366 kB)
  Using cached spacy-2.3.7-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.4 MB)
  Using cached gensim-3.8.3-cp37-cp37m-manylinux1_x86_64.whl (24.2 MB)
  Using cached pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Using cached Unidecode-1.3.2-py3-none-any.whl (235 kB)
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Using cached wasabi-0.9.0-py3-none-any.whl (25 kB)
  Using cached preshed-3.0.6-cp37-c

In [4]:
import string
import re
from pathlib import Path
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('cats')
stop_words = stopwords.words('english')
import boto3
import sagemaker
from sagemaker import get_execution_role
import pandas as pd
import time
import glob
import mapply
from sklearn.model_selection import train_test_split
import json
import datetime
# import texthero as hero

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
mapply.init(
    n_workers=-1,
    chunk_size=1,
    max_chunks_per_worker=8,
    progressbar=True
)

In [5]:
# Secrets
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name="eu-west-1"
)
sagemaker_session = sagemaker.Session(session)

In [6]:
contractions_dict = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because",
    "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not",
    "doesn’t": "does not", "don't": "do not", "don’t": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not",
    "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
    "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
    "i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
    "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not","might've": "might have",
    "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have",
    "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
    "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not",
    "shouldn't've": "should not have", "so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have",
    "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "they'd": "they would",
    "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
    "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
    "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
    "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
    "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is",
    "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
    "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y’all": "you all", "y'all'd": "you all would",
    "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have",
    "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "ain’t": "am not", "aren’t": "are not",
    "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not", "couldn’t’ve": "could not have",
    "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
    "hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have",
    "he’s": "he is", "how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
    "i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would", "it’d’ve": "it would have",
    "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us", "ma’am": "madam", "mayn’t": "may not",
    "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
    "mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock",
    "oughtn’t": "ought not", "oughtn’t’ve": "ought not have", "shan’t": "shall not", "sha’n’t": "shall not", "shan’t’ve": "shall not have",
    "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will", "she’ll’ve": "she will have", "she’s": "she is",
    "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have", "so’ve": "so have", "so’s": "so is",
    "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would", "there’d’ve": "there would have",
    "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will", "they’ll’ve": "they will have",
    "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would", "we’d’ve": "we would have",
    "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
    "what’ll’ve": "what will have", "what’re": "what are", "what’s": "what is", "what’ve": "what have", "when’s": "when is",
    "when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will",
    "who’ll’ve": "who will have", "who’s": "who is", "who’ve": "who have","why’s": "why is", "why’ve": "why have", "will’ve": "will have",
    "won’t": "will not", "won’t’ve": "will not have", "would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have",
    "y’all": "you all", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have", "y’all’re": "you all are",
    "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
    "you’re": "you are", "you’re": "you are", "you’ve": "you have"
}
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function expand the contractions if there's any
def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [7]:
def preprocess_titles_data(my_str):
    my_str = my_str.lower()
    
    # Replacing the repeating pattern of &#039;
    my_str = my_str.replace("&#039;", "")
    
    # Removing all the special Characters
    my_str = my_str.replace(r'[^\w\d\s]',' ')
    
    # Removing all the non ASCII characters
    my_str = my_str.replace(r'[^\x00-\x7F]+',' ')
    
    # Removing the leading and trailing Whitespaces
    my_str = my_str.replace(r'^\s+|\s+?$','')
    
    # Replacing multiple Spaces with Single Space
    my_str = my_str.replace(r'\s+',' ')
    
    # Replacing Two or more dots with one
    my_str = my_str.replace(r'\.{2,}', ' ')
    
    # Expand contradictions
    my_str = expand_contractions(my_str)
    
    my_str = "".join([char for char in my_str if char not in string.punctuation])
    words = word_tokenize(my_str)
    filtered  = [word for word in words if word not in stop_words]
    lemmatize = [lemmatizer.lemmatize(word) for word in filtered]
    return ' '.join(lemmatize)

In [8]:
def preprocess_review_text(my_str):
    my_str = my_str.lower()
    # Replacing the repeating pattern of &#039;
    my_str = my_str.replace("&#039;", "")    
    # Removing all the special Characters
    my_str = my_str.replace(r'[^\w\d\s]',' ')    
    # Removing all the non ASCII characters
    my_str = my_str.replace(r'[^\x00-\x7F]+',' ')    
    # Removing the leading and trailing Whitespaces
    my_str = my_str.replace(r'^\s+|\s+?$','')    
    # Replacing multiple Spaces with Single Space
    my_str = my_str.replace(r'\s+',' ')   
    # Replacing Two or more dots with one
    my_str = my_str.replace(r'\.{2,}', ' ')
    # Expand contradictions
    my_str = expand_contractions(my_str)
    my_str = "".join([char for char in my_str if char not in string.punctuation])
    
    words = word_tokenize(my_str)
    filtered  = [word for word in words if word not in stop_words]
    lemmatize = [lemmatizer.lemmatize(word) for word in filtered]
    return ' '.join(lemmatize)

# Preprocess the data and Dataset preparation

In [9]:
test = pd.read_csv("/root/projects/MMIT/MMIT_message_monitor/data/drugsComTest_raw.tsv", sep = '\t')
train = pd.read_csv("/root/projects/MMIT/MMIT_message_monitor/data/drugsComTrain_raw.tsv", sep = '\t')

In [10]:
test.shape

(53766, 7)

In [11]:
test = test[['review', 'rating']]
train = train[['review', 'rating']]

In [12]:
train['review'] = train['review'].mapply(lambda x: x.strip('"'))
test['review'] = test['review'].mapply(lambda x: x.strip('"'))

In [13]:
train['rating'].value_counts()

10.0    50989
9.0     27531
1.0     21619
8.0     18890
7.0      9456
5.0      8013
2.0      6931
3.0      6513
6.0      6343
4.0      5012
Name: rating, dtype: int64

In [14]:
test['rating'].value_counts()

10.0    17016
9.0      9177
1.0      7299
8.0      6156
7.0      3091
5.0      2710
2.0      2334
3.0      2205
6.0      2119
4.0      1659
Name: rating, dtype: int64

In [15]:
# train['review_cleaned'] = train['review'].mapply(lambda x: preprocess_review_text(x))
# test['review_cleaned'] = test['review'].mapply(lambda x: preprocess_review_text(x))

In [16]:
train['review_cleaned'] = hero.clean(train['review'])
test['review_cleaned'] = hero.clean(test['review'])

In [17]:
train.head()

,review,rating,review_cleaned
0,"It has no side effect, I take it in combinatio...",9.0,side effect take combination bystolic mg fish oil
1,My son is halfway through his fourth week of I...,8.0,son halfway fourth week intuniv became concern...
2,"I used to take another oral contraceptive, whi...",5.0,used take another oral contraceptive pill cycl...
3,This is my first time using any form of birth ...,8.0,first time using form birth control glad went ...
4,Suboxone has completely turned my life around....,9.0,suboxone completely turned life around feel he...


In [18]:
def new_rating(x):
    if x < 4:
        return 0
    elif x > 3 and x < 8:
        return 1
    elif x > 7:
        return 2  

In [19]:
train['new_rating'] = train['rating'].mapply(lambda x: new_rating(x))
test['new_rating'] = test['rating'].mapply(lambda x: new_rating(x))

In [20]:
print(train['new_rating'].value_counts())
print(test['new_rating'].value_counts())

2    97410
0    35063
1    28824
Name: new_rating, dtype: int64
2    32349
0    11838
1     9579
Name: new_rating, dtype: int64


In [21]:
print(train[train['review_cleaned'] == ''].index)
print(test[test['review_cleaned'] == ''].index)

Int64Index([10605, 42909, 113792, 123174, 128652, 143619], dtype='int64')
Int64Index([16938, 21617], dtype='int64')


In [22]:
train = train[train.review_cleaned != '']
test = test[test.review_cleaned != '']

In [23]:
print(train[train['review_cleaned'] == ''].index)
print(test[test['review_cleaned'] == ''].index)

Int64Index([], dtype='int64')
Int64Index([], dtype='int64')


In [24]:
train.head()

,review,rating,review_cleaned,new_rating
0,"It has no side effect, I take it in combinatio...",9.0,side effect take combination bystolic mg fish oil,2
1,My son is halfway through his fourth week of I...,8.0,son halfway fourth week intuniv became concern...,2
2,"I used to take another oral contraceptive, whi...",5.0,used take another oral contraceptive pill cycl...,1
3,This is my first time using any form of birth ...,8.0,first time using form birth control glad went ...,2
4,Suboxone has completely turned my life around....,9.0,suboxone completely turned life around feel he...,2


In [25]:
train.to_csv('data_fst/train_preprocessed.csv')
test.to_csv('data_fst/test_preprocessed.csv')

In [26]:
train, dev = train_test_split(train, test_size=0.25, random_state=0)

# Dataset preparation

In [27]:
def transform_instance_json(row):
    cur_row = dict()
    if len(row['review_cleaned']) < 100000:
        cur_row['source'] = row['review_cleaned']
        cur_row['labels'] = row['new_rating']
    return cur_row

In [28]:
output_train = 'clean_review_new_rating.train'
output_valid = 'clean_review_new_rating.validation'
output_test = 'clean_review_new_rating.test'

In [29]:
all_rows = []
for idx, row in train.iterrows():
    all_rows.append(transform_instance_json(row))
print(len(all_rows))
with open(output_train, 'w') as f:
    for x in all_rows:
        json.dump(x, f) 
        f.write("\n")

all_rows = []
for idx, row in dev.iterrows():
    all_rows.append(transform_instance_json(row))
print(len(all_rows))
with open(output_valid, 'w') as f:
    for x in all_rows:
        json.dump(x, f) 
        f.write("\n")
        
all_rows = []
for idx, row in test.iterrows():
    all_rows.append(transform_instance_json(row))
print(len(all_rows))
with open(output_test, 'w') as f:
    for x in all_rows:
        json.dump(x, f) 
        f.write("\n")

120968
40323
53764


In [30]:
sagemaker_session.upload_data(
    path=output_train,
    bucket='evaluate-general',
    key_prefix='MMIT_message_monitor/sentiment_analysis/final_datasets'
)
sagemaker_session.upload_data(
    path=output_valid,
    bucket='evaluate-general',
    key_prefix='MMIT_message_monitor/sentiment_analysis/final_datasets'
)
sagemaker_session.upload_data(
    path=output_test,
    bucket='evaluate-general',
    key_prefix='MMIT_message_monitor/sentiment_analysis/final_datasets'
)

's3://evaluate-general/MMIT_message_monitor/sentiment_analysis/final_datasets/clean_review_new_rating.test'

In [31]:
def transform_instance_json(row):
    cur_row = dict()
    if len(row['review_cleaned']) < 100000:
        cur_row['source'] = row['review_cleaned']
        cur_row['labels'] = row['rating']
    return cur_row

In [32]:
output_train = 'clean_review_rating.train'
output_valid = 'clean_review_rating.validation'
output_test = 'clean_review_rating.test'

In [33]:
all_rows = []
for idx, row in train.iterrows():
    all_rows.append(transform_instance_json(row))
print(len(all_rows))
with open(output_train, 'w') as f:
    for x in all_rows:
        json.dump(x, f) 
        f.write("\n")

all_rows = []
for idx, row in dev.iterrows():
    all_rows.append(transform_instance_json(row))
print(len(all_rows))
with open(output_valid, 'w') as f:
    for x in all_rows:
        json.dump(x, f) 
        f.write("\n")
        
all_rows = []
for idx, row in test.iterrows():
    all_rows.append(transform_instance_json(row))
print(len(all_rows))
with open(output_test, 'w') as f:
    for x in all_rows:
        json.dump(x, f) 
        f.write("\n")

120968
40323
53764


In [34]:
sagemaker_session.upload_data(
    path=output_train,
    bucket='evaluate-general',
    key_prefix='MMIT_message_monitor/sentiment_analysis/final_datasets'
)
sagemaker_session.upload_data(
    path=output_valid,
    bucket='evaluate-general',
    key_prefix='MMIT_message_monitor/sentiment_analysis/final_datasets'
)
sagemaker_session.upload_data(
    path=output_test,
    bucket='evaluate-general',
    key_prefix='MMIT_message_monitor/sentiment_analysis/final_datasets'
)

's3://evaluate-general/MMIT_message_monitor/sentiment_analysis/final_datasets/clean_review_rating.test'

In [35]:
def transform_instance_json(row):
    cur_row = dict()
    if len(row['review']) < 100000:
        cur_row['source'] = row['review']
        cur_row['labels'] = row['new_rating']
    return cur_row

In [36]:
output_train = 'review_new_rating.train'
output_valid = 'review_new_rating.validation'
output_test = 'review_new_rating.test'

In [37]:
all_rows = []
for idx, row in train.iterrows():
    all_rows.append(transform_instance_json(row))
print(len(all_rows))
with open(output_train, 'w') as f:
    for x in all_rows:
        json.dump(x, f) 
        f.write("\n")

all_rows = []
for idx, row in dev.iterrows():
    all_rows.append(transform_instance_json(row))
print(len(all_rows))
with open(output_valid, 'w') as f:
    for x in all_rows:
        json.dump(x, f) 
        f.write("\n")
        
all_rows = []
for idx, row in test.iterrows():
    all_rows.append(transform_instance_json(row))
print(len(all_rows))
with open(output_test, 'w') as f:
    for x in all_rows:
        json.dump(x, f) 
        f.write("\n")

120968
40323
53764


In [38]:
sagemaker_session.upload_data(
    path=output_train,
    bucket='evaluate-general',
    key_prefix='MMIT_message_monitor/sentiment_analysis/final_datasets'
)
sagemaker_session.upload_data(
    path=output_valid,
    bucket='evaluate-general',
    key_prefix='MMIT_message_monitor/sentiment_analysis/final_datasets'
)
sagemaker_session.upload_data(
    path=output_test,
    bucket='evaluate-general',
    key_prefix='MMIT_message_monitor/sentiment_analysis/final_datasets'
)

's3://evaluate-general/MMIT_message_monitor/sentiment_analysis/final_datasets/review_new_rating.test'

# Run Blazingtext

In [39]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

  
# Secrets
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name="eu-west-1"
)
sagemaker_session = sagemaker.Session(session)

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

arn:aws:iam::200459656367:role/service-role/AmazonSageMaker-ExecutionRole-20210115T135625


In [40]:
s3_train_data = 's3://evaluate-general/MMIT_message_monitor/sentiment_analysis/final_datasets/clean_review_new_rating.train'
s3_validation_data = 's3://evaluate-general/MMIT_message_monitor/sentiment_analysis/final_datasets/clean_review_new_rating.validation'

In [41]:
s3_output_location = 's3://evaluate-general/MMIT_message_monitor/sentiment_analysis/final_datasets/output'

In [42]:
region_name = boto3.Session().region_name
container = sagemaker.amazon.amazon_estimator.get_image_uri('eu-west-1', "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


Using SageMaker BlazingText container: 685385470294.dkr.ecr.eu-west-1.amazonaws.com/blazingtext:1 (eu-west-1)


In [43]:
# Create a model object set to using "Pipe" mode.
mc_model = sagemaker.estimator.Estimator(container,
                                      role,
                                      instance_count=1,
                                      train_instance_type='ml.m5.xlarge',
                                      train_volume_size = 100,
                                      input_mode = 'Pipe',
                                      output_path=s3_output_location,
                                      sagemaker_session=sagemaker_session)

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [44]:
mc_model.set_hyperparameters(mode="supervised",
                            epochs=15,
                            min_count=2,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=3)

In [45]:
train_data = sagemaker.session.s3_input(s3_train_data,
                                        distribution='FullyReplicated',
                                        content_type='application/x-recordio',
                                        s3_data_type='AugmentedManifestFile',
                                        attribute_names=['source', 'labels'],
                                        input_mode='Pipe',
                                        record_wrapping='RecordIO') 

validation_data = sagemaker.session.s3_input(s3_validation_data,
                                        distribution='FullyReplicated',
                                        content_type='application/x-recordio',
                                        s3_data_type='AugmentedManifestFile',
                                        attribute_names=['source', 'labels'],
                                        input_mode='Pipe',
                                        record_wrapping='RecordIO') 

data_channels = {'train': train_data, 'validation': validation_data}

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [46]:
mc_model.fit(inputs=data_channels, logs=True)

2022-01-18 10:33:54 Starting - Starting the training job...
2022-01-18 10:34:16 Starting - Launching requested ML instancesProfilerReport-1642502033: InProgress
...
2022-01-18 10:34:51 Starting - Preparing the instances for training.........
2022-01-18 10:36:17 Downloading - Downloading input data...
2022-01-18 10:36:44 Training - Training image download completed. Training in progress.Arguments: train
[01/18/2022 10:36:45 WARNING 140536472942208] Loggers have already been setup.
[01/18/2022 10:36:45 WARNING 140536472942208] Loggers have already been setup.
[01/18/2022 10:36:45 INFO 140536472942208] nvidia-smi took: 0.025252580642700195 secs to identify 0 gpus
[01/18/2022 10:36:45 INFO 140536472942208] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
Read 5M words
Number of words:  30207
Built the vocabulary using PIPE stream
Initialized producer thread..
Loading validation data from PIPE channel:/opt/ml/input/data/val

# Simple Transformers

In [2]:
!pip install mapply
!pip install simpletransformers
!pip install transformers -U
!pip install torchvision
!pip install tensorboardx
!pip install pandas -U

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached mapply-0.1.9-py3-none-any.whl (7.8 kB)
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 248 kB 24.5 MB/s            
  Using cached regex-2022.1.18-cp37-cp37m-manylinux_2_17_x86_

In [1]:
from simpletransformers.classification import ClassificationModel
import wandb
import logging
from statistics import mean
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
pd.__version__

'1.3.5'

In [3]:
train = pd.read_csv("data_fst/train_preprocessed.csv")
test = pd.read_csv("data_fst/test_preprocessed.csv")

In [4]:
train.head()

,Unnamed: 0,review,rating,review_cleaned,new_rating
0,0,"It has no side effect, I take it in combinatio...",9.0,side effect take combination bystolic mg fish oil,2
1,1,My son is halfway through his fourth week of I...,8.0,son halfway fourth week intuniv became concern...,2
2,2,"I used to take another oral contraceptive, whi...",5.0,used take another oral contraceptive pill cycl...,1
3,3,This is my first time using any form of birth ...,8.0,first time using form birth control glad went ...,2
4,4,Suboxone has completely turned my life around....,9.0,suboxone completely turned life around feel he...,2


In [5]:
# def new_rating(x):
#     if x == 'negative':
#         return 0
#     elif x == 'neutral':
#         return 1
#     elif x == 'positive':
#         return 2

In [6]:
# train['new_rating'] = train['new_rating'].apply(lambda x: new_rating(x))
# test['new_rating'] = test['new_rating'].apply(lambda x: new_rating(x))

In [7]:
train_df = train[['review_cleaned', 'new_rating']]
train_df.columns = ['text', 'label']

train_df, eval_df = train_test_split(train_df, test_size=0.25, random_state=0)

test_df = test[['review_cleaned', 'new_rating']]
test_df.columns = ['text', 'label']

In [8]:
print(train_df.shape, eval_df.shape, test_df.shape)

(120968, 2) (40323, 2) (53764, 2)


In [9]:
train_df.head()

,text,label
102131,really like pill several types birth control o...,2
41784,started birth control took relieve horrible pe...,1
66602,took restavit tablets last night 10pm fell asl...,1
115337,one week 75mg day 3months part always clear sk...,1
60319,3rd time past months quot pills day week quot ...,2


In [10]:
frames = []
classes = train_df.label.unique()
sample_size = 10000

for i in classes:
    g = train_df[train_df.label == i].sample(sample_size)
    frames.append(g)

new_train_df = pd.concat(frames)

In [11]:
frames = []
classes = eval_df.label.unique()
sample_size = 5000

for i in classes:
    g = eval_df[eval_df.label == i].sample(sample_size)
    frames.append(g)

new_eval_df = pd.concat(frames)

In [12]:
frames = []
classes = test_df.label.unique()
sample_size = 5000

for i in classes:
    g = test_df[test_df.label == i].sample(sample_size)
    frames.append(g)

new_test_df = pd.concat(frames)

In [13]:
print(new_train_df['label'].value_counts(), new_eval_df['label'].value_counts(), new_test_df['label'].value_counts())

2    10000
1    10000
0    10000
Name: label, dtype: int64 2    5000
1    5000
0    5000
Name: label, dtype: int64 2    5000
1    5000
0    5000
Name: label, dtype: int64


In [14]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [15]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

True


In [16]:
# Create a TransformerModel with modified attributes
model = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels=3, use_cuda=cuda_available, args={
                            'learning_rate': 1e-5, 'num_train_epochs': 10, 'reprocess_input_data': True, 'overwrite_output_dir': True})

# Train the model
model.train_model(
    new_train_df,
    eval_df=new_eval_df,
    accuracy=lambda truth, predictions: accuracy_score(
        truth, [round(p) for p in predictions]
    ),
)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/30000 [00:00<?, ?it/s]

Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/opt/conda/lib

KeyboardInterrupt: 

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(
    new_test_df, verbose=True)

In [25]:
print(result)
model_outputs

{'mcc': 0.5687740141128755, 'eval_loss': 0.8718097717285156}


array([[-2.6484375 ,  2.19726562,  1.56933594],
       [-2.26171875, -1.12207031,  3.2890625 ],
       [-3.16210938, -0.09936523,  3.79101562],
       ...,
       [ 2.76953125,  0.58349609, -3.42382812],
       [ 1.82714844,  1.39746094, -3.0546875 ],
       [ 3.5859375 , -1.52441406, -2.77539062]])

# Fast-BERT

In [1]:
!pip install fast-bert
!pip install transformers -U
!pip install torchvision
!pip install tensorboardx
!pip install pandas -U

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached transformers-4.15.0-py3-none-any.whl (3.4 MB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_

In [5]:
train = pd.read_csv("data_fst/train.csv").drop(['Unnamed: 0'], axis = 1)
test = pd.read_csv("data_fst/test.csv").drop(['Unnamed: 0'], axis = 1)
val = pd.read_csv("data_fst/test.csv").drop(['Unnamed: 0'], axis = 1)

In [6]:
train.label.unique()

array([0, 2, 1])

In [7]:
frames = []
classes = train.label.unique()
sample_size = 10000

for i in classes:
    g = train[train.label == i].sample(sample_size)
    frames.append(g)

new_train_df = pd.concat(frames)

In [8]:
frames = []
classes = val.label.unique()
sample_size = 3000

for i in classes:
    g = val[val.label == i].sample(sample_size)
    frames.append(g)

new_eval_df = pd.concat(frames)

In [9]:
frames = []
classes = test.label.unique()
sample_size = 3000

for i in classes:
    g = test[test.label == i].sample(sample_size)
    frames.append(g)

new_test_df = pd.concat(frames)

In [10]:
new_train_df.to_csv("data_fst/new_train.csv", index = False)
new_test_df.to_csv("data_fst/new_test.csv", index = False)
new_eval_df.to_csv("data_fst/new_val.csv", index = False)

In [12]:
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging
import torch

logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

In [13]:
from fast_bert.data_cls import BertDataBunch

DATA_PATH = '/root/projects/MMIT/MMIT_message_monitor/notebook/data_fst'
LABEL_PATH = '/root/projects/MMIT/MMIT_message_monitor/notebook/data_fst'

databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='roberta-base',
                          train_file='new_train.csv',
                          val_file='new_val.csv',
                          label_file='labels.csv',
                          text_col='text',
                          label_col='label',
                          batch_size_per_gpu=16,
                          max_seq_length=512,
                          multi_gpu=True,
                          multi_label=False,
                          model_type='roberta')

Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors


In [14]:
OUTPUT_DIR = '/root/projects/MMIT/MMIT_message_monitor/notebook/data_fst'

learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='roberta-base',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir=OUTPUT_DIR,
						finetuned_wgts_path=None,
						warmup_steps=500,
						multi_gpu=True,
						is_fp16=True,
						multi_label=False,
						logging_steps=50)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [16]:
learner.fit(epochs=6,
			lr=6e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_cosine",
			optimizer_type="lamb")

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


KeyboardInterrupt: 

In [ ]:
learner